# Project Documentation Generator: SQLPage Workout Logger

## Introduction

This Jupyter Notebook is an automated tool designed to generate a comprehensive suite of documentation for the **SQLPage Workout Logger** project. It inspects the project's database, file structure, and source code comments to produce a set of up-to-date markdown reports, fully formatted with YAML frontmatter for use in Obsidian.

The notebook generates the following reports:
1.  **Database Schema Report:** A detailed look at each table, its columns, and sample data.
2.  **Folder Tree Report:** An ASCII representation of the project's file structure.
3.  **SQL Comment Documentation:** A reference guide generated from Javadoc-style comments in your `.sql` files.
4.  **Project Style Guide:** A static document outlining coding and documentation standards.

---

## Table of Contents
1.  [**Configuration**](#1-Configuration)
2.  [**How to Use**](#2-How-to-Use)
3.  [**Report Generation**](#3-Report-Generation)
    - [Part A: Database Schema Report](#Part-A-Database-Schema-Report)
    - [Part B: Folder Tree Report](#Part-B-Folder-Tree-Report)
    - [Part C: SQL Documentation Report](#Part-C-SQL-Documentation-Report)
    - [Part D: Project Style Guide](#Part-D-Project-Style-Guide)

---

## 1. Configuration

All user-configurable settings are located in the first code cell. You must review and update these paths before running the notebook.

- **`project_name`**: The base name used in report titles and filenames (e.g., `"SQLPage - Workout Logger"`).
- **`db_path`**: The full or relative path to the SQLite database file (e.g., `'workouts.db'`).
- **`source_folder_for_tree`**: The path to the project's root folder (e.g., `'./sqlpage'`), used for generating the folder tree and parsing SQL comments.
- **`SAMPLE_ROWS`**: The number of sample data rows to include in the schema report.
- **`export_folder`**: The primary destination folder for the generated markdown reports.
- **`obsidian_folder`**: An optional second destination folder, such as an Obsidian vault. Leave empty (`""`) to disable.

---

## 2. How to Use

1.  Update the variables in the **Configuration** code cell immediately following this section.
2.  Run all cells in the notebook sequentially from top to bottom. Using your editor's "Run All" command is recommended.
3.  The final report files will be saved to the location(s) specified in `export_folder` and `obsidian_folder`.

---

## 3. Report Generation

The subsequent cells perform the automated documentation tasks.

In [62]:
# !pip install pyyaml

In [63]:
## Cell 1: Import all the things! (necessary libraries)
import sqlite3
import os
import datetime
from IPython.display import display, Markdown
import re
from collections import defaultdict
import yaml

In [64]:
## Cell 2: Configuration
# This cell holds all the settings for your reports.

# --- CORE CONFIGURATION ---
# The project name used in titles and filenames
project_name = "SQLPage - Workout"

# Path to the database file to document
db_path = '/Volumes/Public/Container_Settings/sqlpage/www/workouts.db'

# Path to the project folder to map into an ASCII tree
source_folder_for_tree = '/Volumes/Public/Container_Settings/sqlpage' 

# --- REPORT CONFIGURATION ---
# Number of sample data rows to include in the schema report
SAMPLE_ROWS = 3

# --- EXPORT CONFIGURATION ---
# The primary folder where reports will be saved (e.g., for GitHub)
export_folder = '/Volumes/Public/Container_Settings/sqlpage/reports'

# Second location to save reports (e.g., your Obsidian vault)
obsidian_folder = "/Users/david/Documents/remote/04 - Coding Project Docs"

# NEW: Subfolder name to use within the Obsidian vault
obsidian_project_subfolder = "SQLPage - Workout Logger"

### Part A: Database Schema
*These cells connect to the database and generate the schema report content.*

In [65]:
# cell 3: YAML configuration
# This cell contains the YAML configuration for the report.
schema_docs = {}
# Add this to your config
schema_yaml_path = '/Volumes/Public/Container_Settings/sqlpage/schema'

if os.path.isdir(schema_yaml_path):
    for filename in os.listdir(schema_yaml_path):
        # print(f"Loading schema documentation from {filename}...")
        if filename.endswith('.yaml'):
            with open(os.path.join(schema_yaml_path, filename), 'r') as f:
                doc = yaml.safe_load(f)
                if 'table_name' in doc:
                    table_name = doc['table_name']
                    schema_docs[table_name] = {
                        'description': doc.get('description', '')}
                    schema_docs[table_name]['columns'] = {
                        col['name']: col.get('description', '')
                        for col in doc.get('columns', []) if isinstance(col, dict) and 'name' in col
                    }
    print(f"✅ Loaded schema documentation for {len(schema_docs)} tables.")
else:
    print("⚠️ Schema YAML directory not found. Descriptions will be missing.")

✅ Loaded schema documentation for 1 tables.


In [66]:
# Cell 4: Fetch Table Names and View Names

table_names = []
view_names = []  # New list to hold view names
error_message = None

if os.path.exists(db_path):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Fetch table names
        cursor.execute(
            "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
        )
        table_names = [table[0] for table in cursor.fetchall()]

        # Fetch view names
        cursor.execute(
            "SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite_%';"
        )
        view_names = [view[0] for view in cursor.fetchall()]

        conn.close()

        # Updated print statement
        print(f"✅ Found {len(table_names)} tables: {', '.join(table_names)}")
        if view_names:
            print(f"✅ Found {len(view_names)} views: {', '.join(view_names)}")

    except sqlite3.Error as e:
        error_message = f"❌ Database error: {e}"
        print(error_message)
else:
    print("Skipping database processing because the path was not found.")

✅ Found 8 tables: sessions, _migrations, DimUser, DimDate, DimExercise, DimUserExercisePreferences, DimExercisePlan, FactWorkoutHistory


In [67]:
## Cell 5: Initialize Report and Connect to Database
# This cell prepares for report generation and opens the database connection.

markdown_parts = []
conn = None  # Initialize conn to None

if table_names or view_names:
    try:
        conn = sqlite3.connect(db_path)
        print("✅ Database connection opened.")
    except sqlite3.Error as e:
        print(f"❌ Failed to connect to database: {e}")
        # conn will remain None on failure
else:
    print("Skipping database report generation as no tables or views were found.")

✅ Database connection opened.


In [68]:
## Cell 6: Generate Table of Contents
# This cell builds the markdown table of contents for the report.

if conn:  # Only proceed if the connection was successful
    markdown_parts.append("\n## Table of Contents\n")

    # Add tables to TOC
    if table_names:
        markdown_parts.append("\n**Tables**\n")
        for table_name in sorted(table_names):
            link_anchor = table_name.lower().replace("_", "-")
            markdown_parts.append(f"- [{table_name}](#{link_anchor})\n")

    # Add views to TOC
    if view_names:
        markdown_parts.append("\n**Views**\n")
        for view_name in sorted(view_names):
            link_anchor = view_name.lower().replace("_", "-")
            markdown_parts.append(f"- [{view_name}](#{link_anchor})\n")

    print("✅ Table of Contents generated.")

✅ Table of Contents generated.


In [69]:
## Cell 7: Generate Database Properties Summary
# This cell adds a high-level summary of the database file's properties.

if conn:
    cursor = conn.cursor()
    
    # Fetch database properties
    cursor.execute("PRAGMA encoding;")
    encoding = cursor.fetchone()[0]
    
    cursor.execute("PRAGMA page_size;")
    page_size = cursor.fetchone()[0]
    
    cursor.execute("PRAGMA foreign_keys;")
    foreign_keys_status = "On" if cursor.fetchone()[0] == 1 else "Off"
    
    cursor.execute("PRAGMA journal_mode;")
    journal_mode = cursor.fetchone()[0].upper()

    cursor.execute("PRAGMA user_version;")
    user_version = cursor.fetchone()[0]

    # Assemble the markdown section
    markdown_parts.append("\n## Database Properties\n\n")
    properties_table = "| Property | Value |\n"
    properties_table += "| :--- | :--- |\n"
    properties_table += f"| Encoding | {encoding} |\n"
    properties_table += f"| Page Size | {page_size} bytes |\n"
    properties_table += f"| Foreign Key Enforcement | {foreign_keys_status} |\n"
    properties_table += f"| Journal Mode | {journal_mode} |\n"
    properties_table += f"| User Version | {user_version} |\n"
    
    markdown_parts.append(properties_table)
    
    print("✅ Database Properties section generated.")

✅ Database Properties section generated.


In [70]:
# Cell 8: Define Helper Functions for Schema Details
# This cell defines a set of modular functions to generate different parts of the schema report.

# --- Individual Formatting Functions ---


def format_column_details(object_name, db_connection, schema_docs):
    """Formats the 'Columns' table for a given object, including descriptions from YAML."""
    parts = ["**Columns**\n\n"]

    # 1. ADDED "Description" to the markdown table header
    parts.append(
        "| Name | Type | Not Null | Default | Primary Key | Description |\n")
    parts.append("| :--- | :--- | :--- | :--- | :--- | :--- |\n")

    cursor = db_connection.cursor()
    cursor.execute(f"PRAGMA table_info('{object_name}');")

    # 2. GET the specific column documentation for this table, with a fallback
    table_column_docs = schema_docs.get(object_name, {}).get('columns', {})

    for col in cursor.fetchall():
        col_name = col[1]
        name_md = f"`{col_name}`"

        # Get the description for this specific column from the loaded docs
        col_desc = table_column_docs.get(col_name, "")

        col_type = col[2]
        not_null = "✅" if col[3] == 1 else ""
        default_val = f"`{col[4]}`" if col[4] is not None else ""
        is_pk = "✅" if col[5] == 1 else ""

        # 4. ADD the col_desc variable to the final table row
        parts.append(
            f"| {name_md} | {col_type} | {not_null} | {default_val} | {is_pk} | {col_desc} |\n"
        )

    parts.append("\n")
    return "".join(parts)


def format_foreign_keys(table_name, db_connection):
    """Formats the 'Foreign Keys' table for a given table."""
    parts = []
    cursor = db_connection.cursor()
    cursor.execute(f"PRAGMA foreign_key_list('{table_name}');")
    foreign_keys = cursor.fetchall()
    if foreign_keys:
        parts.append("**Foreign Keys**\n\n")
        parts.append("| Column | References Table | Foreign Column |\n")
        parts.append("| :----- | :--------------- | :------------- |\n")
        for fk in foreign_keys:
            parts.append(f"| `{fk[3]}` | `{fk[2]}` | `{fk[4]}` |\n")
        parts.append("\n")
    return "".join(parts)


def format_indexes(table_name, db_connection):
    """Formats the 'Indexes' table for a given table."""
    parts = []
    cursor = db_connection.cursor()
    cursor.execute(f"PRAGMA index_list('{table_name}');")
    indexes = cursor.fetchall()
    if indexes:
        parts.append("**Indexes**\n\n")
        parts.append("| Index Name | Columns | Unique |\n")
        parts.append("| :--- | :--- | :--- |\n")
        for index in indexes:
            index_name = f"`{index[1]}`"
            is_unique = "✅" if index[2] == 1 else ""
            cursor.execute(f"PRAGMA index_info('{index[1]}');")
            indexed_columns = [f"`{info[2]}`" for info in cursor.fetchall()]
            columns_str = ", ".join(indexed_columns)
            parts.append(f"| {index_name} | {columns_str} | {is_unique} |\n")
        parts.append("\n")
    return "".join(parts)


def format_triggers(table_name, db_connection):
    """Formats the 'Triggers' section for a given table."""
    parts = []
    cursor = db_connection.cursor()
    cursor.execute(
        f"SELECT name, sql FROM sqlite_master WHERE type='trigger' AND tbl_name='{table_name}';"
    )
    triggers = cursor.fetchall()
    if triggers:
        parts.append("**Triggers**\n\n")
        for trigger_name, trigger_sql in triggers:
            parts.append(f"**Trigger:** `{trigger_name}`\n")
            parts.append(f"```sql\n{trigger_sql}\n```\n")
        parts.append("\n")
    return "".join(parts)


def format_creation_sql(object_name, object_type_title, db_connection):
    """Formats the 'Creation SQL' section for a given object."""
    parts = []
    cursor = db_connection.cursor()
    db_object_type = object_type_title.lower()
    cursor.execute(
        f"SELECT sql FROM sqlite_master WHERE type='{db_object_type}' AND name='{object_name}';"
    )
    result = cursor.fetchone()
    if result and result[0]:
        parts.append("**Creation SQL**\n\n")
        parts.append(f"```sql\n{result[0]}\n```\n")
    return "".join(parts)


def format_data_samples(object_name, db_connection, num_rows):
    """Formats the 'Data Samples' table for a given object."""
    parts = [f"\n**Data Samples (First {num_rows} Rows)**\n\n"]
    cursor = db_connection.cursor()
    try:
        cursor.execute(f'SELECT * FROM "{object_name}" LIMIT {num_rows};')
        col_headers = [desc[0] for desc in cursor.description]
        sample_data = cursor.fetchall()
        if not sample_data:
            parts.append("_Object is empty._\n\n")
        else:
            parts.append(f"| {' | '.join(col_headers)} |\n")
            parts.append(f"|{' :--- |' * len(col_headers)}\n")
            for row in sample_data:
                processed_row = [
                    str(item) if item is not None else "NULL" for item in row
                ]
                parts.append(f"| {' | '.join(processed_row)} |\n")
            parts.append("\n")
    except sqlite3.Error as e:
        parts.append(f"_Could not retrieve data: {e}_\n\n")
    return "".join(parts)


# --- Main Orchestrator Function (with Error Handling) ---

def generate_object_details(object_name, object_type_title, db_connection, schema_docs):
    """
    Fetches and formats all details for a database object by calling specialized functions.
    Catches errors for invalid or temporary objects.
    """
    try:
        details_parts = [f"\n---\n### {object_name}\n\n"]

        # Get the description. If the table or the description key
        # doesn't exist, `table_desc` will be `None`, which is safe to check.
        table_desc = schema_docs.get(object_name, {}).get('description')
        if table_desc:
            details_parts.append(f"_{table_desc}_\n\n")

        # Pass schema_docs down to the next function
        details_parts.append(format_column_details(
            object_name, db_connection, schema_docs))

        if object_type_title == "Table":
            details_parts.append(format_foreign_keys(
                object_name, db_connection))
            details_parts.append(format_indexes(object_name, db_connection))
            details_parts.append(format_triggers(object_name, db_connection))

        details_parts.append(
            format_creation_sql(object_name, object_type_title, db_connection)
        )
        details_parts.append(
            format_data_samples(object_name, db_connection, SAMPLE_ROWS)
        )

        return "".join(details_parts)
    except sqlite3.OperationalError as e:
        # Gracefully handle errors for specific objects without crashing
        print(
            f"⚠️  Skipping object '{object_name}' due to a processing error: {e}")
        error_report = [
            f"\n---\n### {object_name}\n\n",
            f"_Could not generate documentation for this object. It may be temporary or invalid._\n\n",
            f"**Error:** `{e}`\n",
        ]
        return "".join(error_report)


print("✅ Modular helper functions are defined.")

✅ Modular helper functions are defined.


In [71]:
# Cell 9: Generate Report Body for Tables and Views
# This cell iterates through tables and views, calling the helper function to build the report content.

if conn:
    # Generate Details for Tables
    if table_names:
        markdown_parts.append("\n## Table Schemas\n")
        for table_name in sorted(table_names):
            # FIX: Pass schema_docs to the function call
            markdown_parts.append(generate_object_details(
                table_name, "Table", conn, schema_docs))

    # Generate Details for Views
    if view_names:
        markdown_parts.append("\n## View Schemas\n")
        for view_name in sorted(view_names):
            # FIX: Pass schema_docs here as well
            # Note: You may decide not to document views with YAML, but passing the variable is harmless.
            markdown_parts.append(generate_object_details(
                view_name, "View", conn, schema_docs))

    print("✅ Report body generated for all tables and views.")

✅ Report body generated for all tables and views.


In [72]:
## Cell 10: Finalize Report and Close Connection
# This cell assembles the final report string and closes the database connection.

schema_report_body = ""
if conn:
    schema_report_body = "".join(markdown_parts)
    conn.close()
    print("✅ Report content finalized and database connection closed.")
else:
    print("Skipping report finalization as there was no database connection.")

✅ Report content finalized and database connection closed.


In [73]:
## Cell 11: Helper Function for Saving
# This cell defines a reusable function to handle saving files to prevent code duplication.


def save_report(folder_path, filename, content):
    """A helper function to safely save content to a file."""
    if not folder_path:
        print("_Skipping save because path is not set._")
        return
    try:
        os.makedirs(folder_path, exist_ok=True)
        full_path = os.path.join(folder_path, filename)
        with open(full_path, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"✅ Report successfully saved to:\n'{full_path}'")
    except Exception as e:
        print(f"❌ Failed to save report to '{folder_path}': {e}")

In [74]:
## Cell 12: Assemble and Save Schema Report
# This cell assembles the final schema document with Obsidian frontmatter and saves it.

if schema_report_body:
    date_iso = datetime.date.today().isoformat()
    
    report_type = "Database Schema Report"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"
    
    summary = f"Schema and data samples for the database powering the {project_name} application."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{db_path}"
categories: Homelab
tags:
  - sqlpage
  - workout
  - database
  - schema
  - documentation
cssclasses:
  - academia
  - academia-rounded
---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""
    obsidian_content = frontmatter + callout + schema_report_body

    # --- Version 2: Content for GitHub/Primary Export (New format) ---
    github_h1_title = f"{date_iso} - {report_type}"
    github_header = f"# {github_h1_title}\\n\\n"
    github_summary = f'**summary:**\\n"{summary}"\\n\\n'
    github_content = github_header + github_summary + callout + "\\n\\n---\\n" + schema_report_body

    # --- SAVING LOGIC ---
    # Save the GitHub version to the primary export folder
    dated_export_folder = os.path.join(export_folder, date_iso)
    save_report(dated_export_folder, filename, github_content)
    
    # If an Obsidian folder is specified, save the original version there
    if obsidian_folder:
        print("\\nSaving copy to Obsidian folder...")
        obsidian_reports_path = os.path.join(obsidian_folder, obsidian_project_subfolder, "reports", date_iso)
        save_report(obsidian_reports_path, filename, obsidian_content)
else:
    print("Skipping save for schema report because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-07-02/2025-07-02 - SQLPage - Workout - Database Schema Report.md'
\nSaving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/SQLPage - Workout Logger/reports/2025-07-02/2025-07-02 - SQLPage - Workout - Database Schema Report.md'


In [83]:
# Cell 13: Generate or Update Schema YAML files

if not os.path.isdir(schema_yaml_path):
    print(f"⚠️ Creating schema directory at: {schema_yaml_path}")
    os.makedirs(schema_yaml_path)

if table_names:
    print("Checking for schema drift (new tables or new columns)...")
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Iterate through all tables found in the database
    for table_name in sorted(table_names):
        yaml_filename = f"{table_name}.yml"
        yaml_filepath = os.path.join(schema_yaml_path, yaml_filename)

        # Get the current list of columns for the table from the database
        cursor.execute(f"PRAGMA table_info('{table_name}');")
        db_columns_info = cursor.fetchall()
        db_column_names = {col[1] for col in db_columns_info}

        # --- CASE 1: The YAML file does NOT exist. Create it from scratch. ---
        if not os.path.exists(yaml_filepath):
            print(
                f"  -> New table found! Creating placeholder file: {yaml_filename}")
            yaml_structure = {
                'table_name': table_name,
                'description': 'ADD A DESCRIPTION FOR THIS TABLE',
                'owner': 'NEEDS OWNER',
                'tags': [],
                'columns': [
                    {'name': col[1], 'description': 'ADD A DESCRIPTION FOR THIS COLUMN', 'tests': [
                    ]}
                    # UPDATED: Sort columns alphabetically when creating a new file
                    for col in sorted(db_columns_info, key=lambda c: c[1])
                ]
            }
            with open(yaml_filepath, 'w', encoding='utf-8') as f:
                yaml.dump(yaml_structure, f, sort_keys=False,
                          default_flow_style=False, indent=2)

        # --- CASE 2: The YAML file EXISTS. Check for new columns and update it. ---
        else:
            with open(yaml_filepath, 'r', encoding='utf-8') as f:
                doc = yaml.safe_load(f) or {}

            if 'columns' not in doc:
                doc['columns'] = []

            yaml_column_names = {col['name'] for col in doc.get(
                'columns', []) if isinstance(col, dict) and 'name' in col}

            new_columns = db_column_names - yaml_column_names

            if new_columns:
                print(
                    f"  -> Updating {yaml_filename} with {len(new_columns)} new column(s).")
                for col_name in sorted(list(new_columns)):
                    doc['columns'].append({
                        'name': col_name,
                        'description': 'ADD A DESCRIPTION FOR THIS NEW COLUMN',
                        'tests': []
                    })

                # UPDATED: Sort the entire list of columns alphabetically by name
                doc['columns'].sort(key=lambda c: c.get('name', ''))

                with open(yaml_filepath, 'w', encoding='utf-8') as f:
                    yaml.dump(doc, f, sort_keys=False,
                              default_flow_style=False, indent=2)

    conn.close()
    print("✨ Schema check complete.")

Checking for schema drift (new tables or new columns)...
✨ Schema check complete.


### Part B: Folder Tree
*This cell generates the ASCII tree from the source folder.*

In [76]:
# Cell 14: Generate Folder Tree
sqlpage_tree_string = ""


def build_tree_recursive(dir_path, prefix=""):
    """A recursive generator function to build the ASCII tree lines."""
    ignore_list = {".DS_Store", "__pycache__", ".git", ".vscode"}
    try:
        items = sorted(
            [item for item in os.listdir(dir_path) if item not in ignore_list]
        )
    except OSError:
        return
    pointers = ["├── "] * (len(items) - 1) + ["└── "]
    for pointer, item in zip(pointers, items):
        yield prefix + pointer + item
        path = os.path.join(dir_path, item)
        if os.path.isdir(path):
            extension = "│   " if pointer == "├── " else "    "
            yield from build_tree_recursive(path, prefix + extension)


if os.path.isdir(source_folder_for_tree):
    root_name = os.path.basename(os.path.abspath(source_folder_for_tree))
    tree_lines = [f"{root_name}/"]
    tree_lines.extend(list(build_tree_recursive(source_folder_for_tree)))
    sqlpage_tree_string = "\n".join(tree_lines)
    print("✅ Successfully generated folder tree string.")
else:
    print(
        f"Skipping tree generation because the source folder '{source_folder_for_tree}' was not found."
    )

✅ Successfully generated folder tree string.


In [77]:
## Cell 15: Assemble and Save Folder Tree Report
# This final cell assembles the folder tree report with frontmatter and saves it.

if sqlpage_tree_string:
    date_iso = datetime.date.today().isoformat()

    report_type = "Folder Tree Report"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"

    summary = f"An ASCII tree representation of the file and folder structure for the {project_name} project."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{source_folder_for_tree}"
categories: Homelab
tags:
  - sqlpage
  - workout
cssclasses:
  - academia
  - academia-rounded

---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""
    tree_report_body = f"## Directory Tree\n\n```tree\n{sqlpage_tree_string}\n```"
    full_content = frontmatter + callout + tree_report_body

    dated_export_folder = os.path.join(export_folder, date_iso)
    save_report(dated_export_folder, filename, full_content)

    if obsidian_folder:
        print("\nSaving copy to Obsidian folder...")
        # NEW: Create the structured path for Obsidian
        obsidian_reports_path = os.path.join(
            obsidian_folder, obsidian_project_subfolder, "reports", date_iso
        )
        save_report(obsidian_reports_path, filename, full_content)
else:
    print("Skipping save for folder tree report because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-07-02/2025-07-02 - SQLPage - Workout - Folder Tree Report.md'

Saving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/SQLPage - Workout Logger/reports/2025-07-02/2025-07-02 - SQLPage - Workout - Folder Tree Report.md'


### Part C: SQL Documentation
*These cells parse the Javadoc-style comments from your `.sql` files.*

In [81]:
## Cell 16: Parse Comments from SQL Files

sql_docs_data = []

javadoc_regex = re.compile(r"/\*\*(.*?)\*/", re.DOTALL)
tag_regex = re.compile(r"@(\w+)\s+(.*?)(?=\s*@\w+|\s*$)", re.DOTALL)

if os.path.isdir(source_folder_for_tree):
    for root, _, files in os.walk(source_folder_for_tree):
        for file in sorted(files):
            if file.endswith(".sql"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()

                    doc_block_match = javadoc_regex.search(content)
                    if doc_block_match:
                        doc_content = doc_block_match.group(1)

                        tags_dict = defaultdict(list)
                        raw_tags = tag_regex.findall(doc_content)

                        for key, val in raw_tags:
                            # Strip whitespace and remove leading asterisks
                            cleaned_val = val.replace("*", "").strip()
                            # If the line starts with a hyphen (for list items), strip it
                            if cleaned_val.startswith("-"):
                                cleaned_val = cleaned_val.lstrip("- ").strip()
                            # Finally, normalize all remaining whitespace
                            cleaned_val = " ".join(cleaned_val.split())

                            tags_dict[key].append(cleaned_val)

                        cleaned_tags = dict(tags_dict)

                        mtime_timestamp = os.path.getmtime(file_path)
                        mtime_date = datetime.datetime.fromtimestamp(
                            mtime_timestamp
                        ).strftime("%Y-%m-%d")
                        cleaned_tags["file_last_modified"] = [mtime_date]

                        cleaned_tags["filepath"] = [file_path]
                        sql_docs_data.append(cleaned_tags)
                except Exception as e:
                    print(f"Could not process file {file_path}: {e}")

print(
    f"✅ Found and parsed documentation comments from {len(sql_docs_data)} SQL files."
)

✅ Found and parsed documentation comments from 30 SQL files.


In [79]:
## Cell 17: Generate SQL Documentation Body
# This cell takes the parsed data from the previous cell and formats it into a readable markdown document body.

sql_docs_body = ""
if sql_docs_data:
    # Sort the data by filepath to ensure a consistent order
    sql_docs_data.sort(key=lambda x: x.get("filepath", [""])[0])

    parts = ["## SQL File Documentation\n"]
    for doc in sql_docs_data:
        # Helper function to safely get the first item from a list in the doc dict
        def get_tag(tag_name, default=""):
            return doc.get(tag_name, [default])[0]

        # --- Header and Metadata ---
        filename = os.path.basename(get_tag("filepath", "N/A"))
        parts.append(f"\n---\n### `{filename}`\n")
        parts.append(f"**Path:** `{get_tag('filepath')}`\n")

        doc_updated = get_tag("last-updated", "N/A")
        file_modified = get_tag("file_last_modified", "N/A")
        date_status_emoji = "✅" if doc_updated == file_modified else "⚠️"
        parts.append(
            f"**Last Updated (doc):** `{doc_updated}` | **File Modified:** `{file_modified}` {date_status_emoji}\n\n"
        )

        # --- Core Description ---
        parts.append(
            f"**Description:** {get_tag('description', '_No description provided._')}\n"
        )

        # --- Function to format list-based tags ---
        def format_list_tag(tag_key, title):
            if tag_key in doc:
                parts.append(f"\n**{title}:**\n")
                for item in doc[tag_key]:
                    parts.append(f"- {item}\n")

        # --- Display all relevant tags in a clean order ---
        format_list_tag("requires", "Requires")
        format_list_tag("param", "Parameters")
        format_list_tag("returns", "Returns")
        format_list_tag("see", "See Also")
        format_list_tag("note", "Notes")
        format_list_tag("todo", "TODO")

    sql_docs_body = "".join(parts)
    print("✅ Successfully generated full SQL documentation body.")

✅ Successfully generated full SQL documentation body.


### Part D: Project Style Guide
*This final cell saves the standalone style guide document.*

In [80]:
## Cell 18: Assemble and Save SQL Documentation Report
# This final cell assembles the full document with frontmatter and saves your new SQL Documentation report.

if sql_docs_body:
    date_iso = datetime.date.today().isoformat()

    report_type = "SQL Comment Documentation"
    title = f"{project_name} - {report_type}"
    filename = f"{date_iso} - {title}.md"

    summary = "A guide to docstring conventions and a summary of all documented SQL files in the project."

    frontmatter = f"""---
date: {date_iso}
title: "{title}"
summary: "{summary}"
series: sqlpage.workout-logger
github: https://github.com/drusho/SQLPage-Workout-Logger
source: "{source_folder_for_tree}"
categories: Homelab
tags:
  - sqlpage
  - documentation
  - style-guide
cssclasses:
  - academia
  - academia-rounded  
---
"""
    callout = f""">[!tip]+ Tip
> - This report was auto-generated using the `SQLPage_Workout_Documentation_Generator.ipynb` notebook.
"""

    toc_parts = ["\n## Table of Contents\n\n"]
    for doc in sql_docs_data:
        filename_in_toc = os.path.basename(doc.get("filepath", ["N/A"])[0])
        # CORRECTED: The .replace('.', '') has been removed to keep the file extension in the link.
        link_anchor = filename_in_toc.lower()
        toc_parts.append(f"- [`{filename_in_toc}`](#{link_anchor})\n")
    toc_body = "".join(toc_parts)

    full_content = frontmatter + callout + toc_body + sql_docs_body

    dated_export_folder = os.path.join(export_folder, date_iso)
    save_report(dated_export_folder, filename, full_content)

    if obsidian_folder:
        print("\nSaving copy to Obsidian folder...")
        obsidian_reports_path = os.path.join(
            obsidian_folder, obsidian_project_subfolder, "reports", date_iso
        )
        save_report(obsidian_reports_path, filename, full_content)
else:
    print("Skipping save for SQL documentation because no content was generated.")

✅ Report successfully saved to:
'/Volumes/Public/Container_Settings/sqlpage/reports/2025-07-02/2025-07-02 - SQLPage - Workout - SQL Comment Documentation.md'

Saving copy to Obsidian folder...
✅ Report successfully saved to:
'/Users/david/Documents/remote/04 - Coding Project Docs/SQLPage - Workout Logger/reports/2025-07-02/2025-07-02 - SQLPage - Workout - SQL Comment Documentation.md'
